In [13]:
import pandas as pd
import numpy as np
import traceback

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

from glob import glob
from tqdm import tqdm
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

# Suppress specific warning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="org.esa.snap.core.dataio.dimap.DimapProductReader")


In [14]:

def check_image(mgrs):
    processed_mosaic = os.listdir("/data/ksa/01_Image_Acquisition/02_Processed_mosaic/")
    if(mgrs in processed_mosaic):
        return True
    else: 
        return False

def get_gc(path_to_sentinel_data):
    # path_to_sentinel_data = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MXU/20230501_20230512.dim"
    product_subset = ProductIO.readProduct(path_to_sentinel_data)
    gc = product_subset.getSceneGeoCoding()
    bands_names = list(product_subset.getBandNames())
    return product_subset,gc,bands_names

def get_values(gc, bands_names, lat, long):
    pixel_pos = gc.getPixelPos(GeoPos(lat, long), None)
    data = list()
    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
            data_values = [float(val) for val in data]
        except Exception as e:
            #print('error occured at',int(pixel_pos.x), int(pixel_pos.y))
            #print(int(pixel_pos.x), int(pixel_pos.y))
            continue
    return data_values

# data_values
def get_value(gc, lat, long):    
    
    data = list()
    pixel_pos = gc.getPixelPos(GeoPos(lat, lon), None)
    data.append(lat).append(lon).append(int(pixel_pos.x)).append(int(pixel_pos.y))

    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
        except Exception as e:
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            #traceback.print_exc()
            continue
    return data


In [15]:
## 32 35 16 63 12 73 18 53 94
## 97  

## edit with province
idprov = "95"

In [16]:
df_ksa = pd.read_csv("/data/raw/processed/cloned_points.csv")
# df_ksa.iloc[:1000].to_csv("cek.csv")
bridng_papua=pd.read_excel('/data/ksa/00_Data_Input/bridging_papua.xlsx')

In [17]:
bridng_papua['KAB2022_2']=bridng_papua.KAB2022_2.astype(str)
bridng_papua['KAB2023_1']=bridng_papua.KAB2023_1.astype(str)

In [18]:
# df_ksa = pd.read_csv("/data/raw/processed/generated_points_unstable.csv")
df_ksa['idprov'] = df_ksa.idsegmen.astype('str').str[:2]
df_ksa['idkab'] = df_ksa.idsegmen.astype('str').str[:4]

In [19]:
df_ksa_khusus=df_ksa.merge(bridng_papua, left_on='idkab',right_on='KAB2022_2',how='inner')
df_ksa_khusus.idsegmen=df_ksa_khusus.apply(lambda y: y.KAB2023_1+str(y.idsegmen)[4:],axis=1)

In [20]:
#df_ksa = pd.read_csv("/data/raw/processed/cloned_points.csv")
# df_ksa.iloc[:1000].to_csv("cek.csv")

# df_ksa = pd.read_csv("/data/raw/processed/generated_points_unstable.csv")
df_ksa_khusus['idprov'] = df_ksa_khusus.idsegmen.astype('str').str[:2]
df_ksa_khusus['idkab'] = df_ksa_khusus.idsegmen.astype('str').str[:4]

df_ksa_khusus['index'] = [x.zfill(2) for x in df_ksa_khusus['index'].astype("str")]
df_ksa_khusus['idpoint'] = df_ksa_khusus.idsegmen.astype('str') + df_ksa_khusus.idsubsegmen.astype('str') + '#' + df_ksa_khusus['index']

df_ksa = df_ksa_khusus.query('idprov == "{0}"'.format(idprov))

df_ksa.shape

(11925, 25)

In [21]:
## list all MGRS
mgrs_all_ = list(df_ksa.MGRS.unique())

## Cek MGRS not done
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]

mgrs_not_done_ = [x for x in mgrs_all_ if x not in mgrs_done_]
mgrs_not_done_

['54LVR', '54LVS', '54MVT', '54MUT']

In [24]:
mgrs_all_

['54LVR', '54LVS', '54MVT', '54MUT']

In [23]:
list_un_done=[]
for mgrs in mgrs_not_done_:
    df_result = pd.DataFrame()
    if(check_image(mgrs)):
        try:
            print(mgrs + " available")
            path_to_mgrs = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/"+mgrs+"/"
            all_sentinel_data = glob(path_to_mgrs+"*.dim")
            df_ksa_mgrs = df_ksa.loc[df_ksa.MGRS == mgrs]
            for sent in tqdm(all_sentinel_data):
                path_to_sentinel_data = sent
                periode = path_to_sentinel_data[-21:-4]
                product_subset,gc,bands_names = get_gc(path_to_sentinel_data)
                bands_names = [col for col in bands_names if 'VH' in col]
                df_ksa_mgrs_tmp = df_ksa_mgrs.copy()
                df_ksa_mgrs_tmp = df_ksa_mgrs_tmp.reset_index(drop=True)
                df_ksa_mgrs_tmp.loc[:, bands_names] = 0.0
                df_ksa_mgrs_tmp.loc[:, 'periode'] =periode  
                # print(bands_names)
                for i,r in df_ksa_mgrs_tmp.iterrows():
                    try:
                        val = get_values(gc, bands_names, r['lat'], r['long'])
                        df_ksa_mgrs_tmp.loc[i,bands_names] = val
                    except:
                        print('error while get the data on',r['lat'], r['long'])
                        list_un_done.append([r['lat'], r['long']])
                        continue
                    # break
                # break
                df_result = pd.concat([df_result,df_ksa_mgrs_tmp])
    
            df_result = df_result[['idpoint','MGRS','Sigma0_VH_db','periode']]
            with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'wb') as f:
                pickle.dump(df_result, f)
            
            print("done sampling for : "+ mgrs)
            # break
        except: 
            print("Error on : "+ mgrs)
    else:
        print(mgrs + " not available")
        continue
    # break

54LVR available


100%|██████████| 60/60 [14:32<00:00, 14.54s/it]


done sampling for : 54LVR
54LVS available


100%|██████████| 60/60 [00:54<00:00,  1.11it/s]


done sampling for : 54LVS
54MVT available


100%|██████████| 60/60 [00:59<00:00,  1.01it/s]


done sampling for : 54MVT
54MUT available


100%|██████████| 60/60 [00:52<00:00,  1.14it/s]

done sampling for : 54MUT


In [7]:
## Cek error mgrs
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]

mgrs_not_done_ = [x for x in mgrs_all_ if x not in mgrs_done_]
mgrs_not_done_

['54LVR',
 '54LVS',
 '53MNS',
 '53MPQ',
 '53MPR',
 '54MVT',
 '54MUT',
 '54MUV',
 '54MVV']

### Cek 

In [9]:
## Cek VH Value Overtime on One Point

filtered_df = df_.query('idpoint == "321201001A1#02"').sort_values('periode')

plt.figure(figsize=(10, 6))
plt.plot(filtered_df['periode'], filtered_df['Sigma0_VH_db'], marker='o')

plt.title('Sigma0_VH_db over Time for idpoint "321201003B2#13"')
plt.xlabel('Periode')
plt.ylabel('Sigma0_VH_db')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels if needed

plt.tight_layout()
plt.show()

NameError: name 'df_' is not defined

In [ ]:
## Open one Sampling Data

mgrs = '49LDM'
with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'rb') as f:
    df_ = pickle.load(f)

df_.head()

In [ ]:
df_.describe()